# Volcado Tabular de Vencimientos Explained con Otros Campos

In [111]:
from datetime import date
import pandas as pd
import requests

## Rutas

In [112]:
rutas = {
    "explained": "https://ms-tesoreria-analytics-staging.interconecta2.cl/v1/settlements/swaps/explained",
    "headers": "https://ms-tesoreria-analytics-staging.interconecta2.cl/operations/all",
}

## SDK

Estas son funciones que utilizan los endpoints adecuados para obtener la data necesaria para el volcado.

In [113]:
def get_explained(fecha: date, ruta: str):
    result = requests.get(f"{ruta}/{fecha}")
    if result.status_code == 200:
        return result.json()
    else:
        return result.status_code

In [114]:
def get_headers(ruta: str):
    result = requests.get(ruta)
    if result.status_code == 200:
        return result.json()
    else:
        return result.status_code

## Procedimiento

Se define la fecha de proceso.

In [115]:
fecha_proceso = date(2025, 7, 11)
f_proc = fecha_proceso.isoformat().replace("-", "")

Se obtienen los explained a fecha de proceso.

In [116]:
explained = get_explained(fecha_proceso, rutas["explained"])

Se obtienen los encabezados de la cartera vigente.

In [117]:
headers = get_headers(rutas["headers"])

Vemos un registro de los explained.

In [118]:
explained[0]

{'dealNumber': '3634',
 'legNumber': 1,
 'tipoPata': 'FIXED_RATE_MCCY',
 'recPay': 'A',
 'fechaInicial': '2025-01-13',
 'fechaFinal': '2025-07-11',
 'fechaPago': '2025-07-11',
 'nocional': 5000000000.0,
 'amortizacion': 0.0,
 'interes': 40274999.99999962,
 'amortEsFlujo': True,
 'flujo': 40274999.99999962,
 'monedaNocional': 'CLP',
 'valorTasa': 0.0162,
 'tipoTasa': 'LinAct360',
 'fechaFixingFx': '2025-07-10',
 'monedaPago': 'USD',
 'indiceFx': 'USDOBS',
 'valorIndiceFx': 948.33,
 'amortizacionMonedaPago': 0.0,
 'interesMonedaPago': 42469.39356553058,
 'flujoMonedaPago': 42469.39356553058,
 'fraccionAgno': '(fechaFinal - fechaInicial) / 360',
 'formulaInteres': 'nocional * valorTasa * fraccionAgno',
 'formulaInteresMonedaPago': 'formulaInteres / valorIndiceFx',
 'formulaAmortizacionMonedaPago': 'amortizacion / valorIndiceFx'}

Se observa que no tiene los campos de portfolio (gestión o contable) ni la estrategia contable. Se los agregamos con la data de los headers.

In [119]:
df_explained = pd.DataFrame.from_dict(explained)

Vemos la estructura de un header.

In [120]:
headers["5000"]

{'trade_date': {'fecha': '2022-11-29'},
 'deal_number': '5000',
 'counterparty': {'name': 'BANCO SANTANDER-SANTIAGO',
  'rut': {'rut': 97036000, 'dv': 'K'},
  'other': None},
 'portfolio': 'BANCA',
 'hedge_accounting': 'COB_TASA_INT_ACTIVOS',
 'product': 'SWAP_MONE',
 'currency_pair': 'CLFCLP',
 'price': None,
 'settlement_mechanism': 'C',
 'other': {'regulatory_portfolio': 'COBERTURA_VALOR_RAZONABLE'}}

In [121]:
def generate_new_columns(row):
    # Notar que headers se toma por closure
    operacion = headers.get(row.dealNumber)
    if operacion is None:
        operacion = {"dummy": 1}
    if operacion.get("other") is None:
        regulatory_portfolio = None
    else:
        regulatory_portfolio = operacion.get("other").get("regulatory_portfolio")
    return pd.Series({
        'portfolio': operacion.get("portfolio"),
        'hedgeAccounting': operacion.get("hedge_accounting"),
        'product': operacion.get("product"),
        'settlementMechanism': operacion.get("settlement_mechanism"),
        'regulatoryPortfolio': regulatory_portfolio,
    })

In [ ]:
df_explained_ok = df_explained.join(df_explained.apply(
    generate_new_columns,
    axis=1,
))
del df_explained_ok["fraccionAgno"]
del df_explained_ok["formulaValorTasaEquivalente"]
del df_explained_ok["formulaInteres"]
# TRY - EXCEPT
del df_explained_ok["formulaInteresMonedaPago"]
del df_explained_ok["formulaAmortizacionMonedaPago"]

In [123]:
df_explained_ok

,dealNumber,legNumber,tipoPata,recPay,fechaInicial,fechaFinal,fechaPago,nocional,amortizacion,interes,...,spread,gearing,numDecimalesTasaEquivalente,codigoIndiceTasa,fxRateIndex,portfolio,hedgeAccounting,product,settlementMechanism,regulatoryPortfolio
0,3634,1,FIXED_RATE_MCCY,A,2025-01-13,2025-07-11,2025-07-11,5.000000e+09,0.0,4.027500e+07,...,NaN,NaN,NaN,NaN,NaN,BANCA,NO,SWAP_ICP,C,COBERTURA_RIESGO_POSICION_BALANCE
1,3634,2,OVERNIGHT_INDEX_MCCY,P,2025-01-13,2025-07-11,2025-07-11,-5.000000e+09,0.0,-1.327038e+05,...,0.000000,1.0,6.0,NaN,NaN,BANCA,NO,SWAP_ICP,C,COBERTURA_RIESGO_POSICION_BALANCE
2,4232,1,OVERNIGHT_INDEX_MCCY,A,2025-01-13,2025-07-11,2025-07-11,4.650000e+09,0.0,1.234145e+05,...,0.000000,1.0,6.0,NaN,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
3,4232,2,FIXED_RATE_MCCY,P,2025-01-13,2025-07-11,2025-07-11,-4.650000e+09,0.0,-1.310951e+08,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
4,4233,1,FIXED_RATE_MCCY,A,2025-01-13,2025-07-11,2025-07-11,3.820000e+09,0.0,1.076954e+08,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
5,4233,2,OVERNIGHT_INDEX_MCCY,P,2025-01-13,2025-07-11,2025-07-11,-3.820000e+09,0.0,-1.013857e+05,...,0.000000,1.0,6.0,NaN,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
6,4693,1,FIXED_RATE_MCCY,A,2025-01-13,2025-07-11,2025-07-11,4.650000e+09,0.0,1.514415e+08,...,NaN,NaN,NaN,NaN,NaN,BANCA,NO,SWAP_ICP,C,COBERTURA_RIESGO_POSICION_BALANCE
7,4693,2,OVERNIGHT_INDEX_MCCY,P,2025-01-13,2025-07-11,2025-07-11,-4.650000e+09,0.0,-1.234145e+05,...,0.000000,1.0,6.0,NaN,NaN,BANCA,NO,SWAP_ICP,C,COBERTURA_RIESGO_POSICION_BALANCE
8,5676,1,FIXED_RATE,A,2025-04-11,2025-07-11,2025-07-11,1.150000e+07,0.0,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,EXLIBOR,NO,SWAP_TASA,C,NEGOCIACION
9,5676,2,FIXED_RATE,P,2025-01-13,2025-07-11,2025-07-11,-1.150000e+07,0.0,-0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,EXLIBOR,NO,SWAP_TASA,C,NEGOCIACION


In [ ]:
#df_explained_ok.to_csv(f"./{f_proc}_explained_plus_plus.csv")

Ahora vamos a convertir la dataframe en una vista por operación.

In [125]:
import numpy as np

Crear una función helper para agregar los datos correctamente

In [126]:
def aggregate_deal_data(group):
    """
    Aggregate deal data from leg-level to deal-level
    """
    # Get deal-level attributes (should be the same for all legs)
    deal_attrs = {
        'DealNumber': group['dealNumber'].iloc[0],
        'product': group['product'].iloc[0],
        'fechaFinal': group['fechaFinal'].iloc[0],
        'settlementMechanism': group['settlementMechanism'].iloc[0],
        'hedgeAccounting': group['hedgeAccounting'].iloc[0],
        'regulatoryPortfolio': group['regulatoryPortfolio'].iloc[0]
    }
    
    # Separate active (A) and passive (P) legs
    active_legs = group[group['recPay'] == 'A']
    passive_legs = group[group['recPay'] == 'P']
    
    # Process active legs
    if len(active_legs) > 0:
        deal_attrs['pataActivaAmortizacion'] = active_legs['amortizacion'].sum()
        deal_attrs['pataActivaMonedaNocional'] = active_legs['monedaNocional'].iloc[0]  # Assuming same currency for all active legs
        # Get fechaPago from lowest legNumber for active legs
        deal_attrs['pataActivaFechaPagoAmortizacion'] = active_legs.loc[active_legs['legNumber'].idxmin(), 'fechaPago']
        # Sum flujoMonedaPago, handling NaN values
        active_flujo_values = active_legs['flujoMonedaPago'].fillna(active_legs['flujo'])
        deal_attrs['pataActivaFlujo'] = active_flujo_values.sum() if not active_flujo_values.isna().all() else np.nan
    else:
        deal_attrs['pataActivaAmortizacion'] = np.nan
        deal_attrs['pataActivaMonedaNocional'] = np.nan
        deal_attrs['pataActivaFechaPagoAmortizacion'] = np.nan
        deal_attrs['pataActivaFlujo'] = np.nan
    
    # Process passive legs
    if len(passive_legs) > 0:
        deal_attrs['pataPasivaAmortizacion'] = passive_legs['amortizacion'].sum()
        deal_attrs['pataPasivaMonedaNocional'] = passive_legs['monedaNocional'].iloc[0]  # Assuming same currency for all passive legs
        # Get fechaPago from lowest legNumber for passive legs
        deal_attrs['pataPasivaFechaPagoAmortizacion'] = passive_legs.loc[passive_legs['legNumber'].idxmin(), 'fechaPago']
        # Sum flujoMonedaPago, handling NaN values
        passive_flujo_values = passive_legs['flujoMonedaPago'].fillna(passive_legs['flujo'])
        deal_attrs['pataPasivaFlujo'] = passive_flujo_values.sum() if not passive_flujo_values.isna().all() else np.nan
    else:
        deal_attrs['pataPasivaAmortizacion'] = np.nan
        deal_attrs['pataPasivaMonedaNocional'] = np.nan
        deal_attrs['pataPasivaFechaPagoAmortizacion'] = np.nan
        deal_attrs['pataPasivaFlujo'] = np.nan
    
    return pd.Series(deal_attrs)

Agrupamos por deal number

In [127]:
df_deal_level = df_explained_ok.groupby('dealNumber').apply(aggregate_deal_data).reset_index(drop=True)

C:\Users\bencl\AppData\Local\Temp\ipykernel_43084\4184180563.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_deal_level = df_explained_ok.groupby('dealNumber').apply(aggregate_deal_data).reset_index(drop=True)


Aplicar conversiones de nombres de producto

In [128]:
# Apply product name conversions
product_mapping = {
    'SWAP_ICP': 'Swap Promedio Cámara'.encode('utf-8').decode('utf-8'),
    'SWAP_MONE': 'Swap Moneda',
    'SWAP_TASA': 'Swap Tasas'
}
df_deal_level['product'] = df_deal_level['product'].map(product_mapping).fillna(df_deal_level['product'])

Y calcular monto neto a compensar en caso de modalidad compensación

In [129]:
df_deal_level['netoACompensar'] = np.nan
mask_compensation = df_deal_level['settlementMechanism'] == 'C'

df_deal_level.loc[mask_compensation, 'netoACompensar'] = (
    df_deal_level.loc[mask_compensation, 'pataActivaFlujo'].fillna(0) + 
    df_deal_level.loc[mask_compensation, 'pataPasivaFlujo'].fillna(0)
)

Y reordenamos las columnas

In [130]:
# Reorder columns to include the new netoACompensar column
column_order = [
    'DealNumber', 'product', 'fechaFinal', 'settlementMechanism', 'hedgeAccounting', 
    'regulatoryPortfolio', 'pataActivaAmortizacion', 'pataActivaMonedaNocional', 
    'pataActivaFechaPagoAmortizacion', 'pataActivaFlujo', 'pataPasivaAmortizacion',
    'pataPasivaMonedaNocional', 'pataPasivaFechaPagoAmortizacion', 'pataPasivaFlujo',
    'netoACompensar'
]

df_deal_level = df_deal_level[column_order]

Y vemos los resultados:

In [131]:
print("Deal-level dataframe shape:", df_deal_level.shape)
print("\nFirst few rows:")
print(df_deal_level.head())

print("\nDataframe info:")
print(df_deal_level.info())

print("\nSample of deal-level data:")
print(df_deal_level.head(10))

Deal-level dataframe shape: (9, 15)

First few rows:
  DealNumber               product  fechaFinal settlementMechanism  \
0       3634  Swap Promedio Cámara  2025-07-11                   C   
1       4232  Swap Promedio Cámara  2025-07-11                   C   
2       4233  Swap Promedio Cámara  2025-07-11                   C   
3       4693  Swap Promedio Cámara  2025-07-11                   C   
4       5676            Swap Tasas  2025-07-11                   C   

  hedgeAccounting                regulatoryPortfolio  pataActivaAmortizacion  \
0              NO  COBERTURA_RIESGO_POSICION_BALANCE                     0.0   
1              NO                        NEGOCIACION                     0.0   
2              NO                        NEGOCIACION                     0.0   
3              NO  COBERTURA_RIESGO_POSICION_BALANCE                     0.0   
4              NO                        NEGOCIACION                     0.0   

  pataActivaMonedaNocional pataActivaFechaPag

In [132]:
df_deal_level

,DealNumber,product,fechaFinal,settlementMechanism,hedgeAccounting,regulatoryPortfolio,pataActivaAmortizacion,pataActivaMonedaNocional,pataActivaFechaPagoAmortizacion,pataActivaFlujo,pataPasivaAmortizacion,pataPasivaMonedaNocional,pataPasivaFechaPagoAmortizacion,pataPasivaFlujo,netoACompensar
0,3634,Swap Promedio Cámara,2025-07-11,C,NO,COBERTURA_RIESGO_POSICION_BALANCE,0.0,CLP,2025-07-11,4.246939e+04,0.0,CLP,2025-07-11,-1.327038e+05,-9.023436e+04
1,4232,Swap Promedio Cámara,2025-07-11,C,NO,NEGOCIACION,0.0,CLP,2025-07-11,1.234145e+05,0.0,CLP,2025-07-11,-1.382379e+05,-1.482340e+04
2,4233,Swap Promedio Cámara,2025-07-11,C,NO,NEGOCIACION,0.0,CLP,2025-07-11,1.135632e+05,0.0,CLP,2025-07-11,-1.013857e+05,1.217750e+04
3,4693,Swap Promedio Cámara,2025-07-11,C,NO,COBERTURA_RIESGO_POSICION_BALANCE,0.0,CLP,2025-07-11,1.596928e+05,0.0,CLP,2025-07-11,-1.234145e+05,3.627830e+04
4,5676,Swap Tasas,2025-07-11,C,NO,NEGOCIACION,0.0,USD,2025-07-11,1.338766e+05,0.0,USD,2025-07-11,-1.428351e+05,-8.958533e+03
5,6237,Swap Moneda,2025-07-11,C,NO,COBERTURA_RIESGO_POSICION_BALANCE,0.0,USD,2025-07-11,0.000000e+00,0.0,CLP,2025-07-11,-8.570838e+07,-8.570838e+07
6,6238,Swap Promedio Cámara,2025-07-11,C,NO,NEGOCIACION,0.0,CLP,2025-07-11,1.184980e+08,0.0,CLP,2025-07-11,-1.207653e+08,-2.267333e+06
7,6873,Swap Tasas,2025-07-11,C,NO,NEGOCIACION,0.0,USD,2025-07-11,1.583705e+05,0.0,USD,2025-07-11,-1.334988e+05,2.487172e+04
8,6875,Swap Promedio Cámara,2025-07-11,C,NO,COBERTURA_RIESGO_POSICION_BALANCE,0.0,CLP,2025-07-11,2.088333e+08,0.0,CLP,2025-07-11,-2.012756e+08,7.557777e+06


In [133]:
#df_deal_level.to_csv(f"./{f_proc}_deal_level.csv", index=False, encoding='utf-8')
#df_deal_level.to_excel(f"./{f_proc}_deal_level.xlsx", index=False)
df_deal_level.to_excel(rf"C:\Users\bencl\OneDrive - palace.cl\Documents\Palace\Banco Internacional\Treasury PM\06 Pruebas\{f_proc}_deal_level.xlsx", index=False)
